## Step-1: Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step-2: Download VisDrone Video dataset from Google Drive

In [14]:
%%time
%%bash
if [ -d VisDroneVideo ]
then
  echo "data already exists"
else 
  mkdir -p VisDroneVideo
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-val.zip
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-test-dev.zip
  unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-train.zip
  # unzip -d VisDroneVideo drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/VisDrone2019-VID-test-challenge.zip
fi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 4.99 s, sys: 972 ms, total: 5.97 s
Wall time: 3min 24s


## Step-3: Convert to Visdrone-YOLO format


In [15]:
import os
from pathlib import Path
import shutil
import cv2
import csv
def _visdrone_video_to_yolov7_image_files(data_folder_dir, yolov7_output_folder_dir):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
    """
    # init paths/folders    
    input_ann_folder = str(Path(data_folder_dir) / "annotations")
    print(input_ann_folder)
    annotation_filepath_list = os.listdir(input_ann_folder)
    output_image_folder = str(Path(yolov7_output_folder_dir)/"images")
    Path(output_image_folder).mkdir(parents=True, exist_ok=True)
    image_size_dict = {}
    for annotation_filename in annotation_filepath_list:
      print(annotation_filename)
      image_filepath_dir = os.path.splitext(str(Path(data_folder_dir) / "sequences"/annotation_filename))[0]
      image_filepath_list = os.listdir(image_filepath_dir)
      i = 0
      for image_name in image_filepath_list:
        yolov7_image_name = os.path.splitext(annotation_filename)[0] + "_" + image_name
        path = shutil.copyfile(image_filepath_dir + "/" + image_name,str(Path(output_image_folder)/yolov7_image_name))
        if i == 0:
          img = cv2.imread(path)
          image_size_dict[annotation_filename] = img.shape
        i += 1
    return image_size_dict    
    



In [16]:
def _convert_visidrone_row_to_yolov7_row(size, box):
    dw = 1./size[1]
    dh = 1./size[0]
    x = int(box[2]) + int(box[4])/2.0
    y = int(box[3]) + int(box[5])/2.0
    w = int(box[4])
    h = int(box[5])
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (box[7], x,y,w,h)

In [17]:
def _visdrone_video_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir, image_size_dict):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
        image_size_dict: dict
            contains the image_size info for various videos
    """
    # init paths/folders    
    input_ann_folder = str(Path(data_folder_dir) / "annotations")
    print(input_ann_folder)
    annotation_filepath_list = os.listdir(input_ann_folder)
    output_annotations_folder = str(Path(yolov7_output_folder_dir)/"labels")
    Path(output_annotations_folder).mkdir(parents=True, exist_ok=True)
    annotation_dict = {}
    for annotation_filename in annotation_filepath_list:
      image_size = image_size_dict[annotation_filename]
      print("image size for (h,w,c) " + annotation_filename + " ==> " + str(image_size))
      with open(Path(input_ann_folder)/annotation_filename) as file_obj:
        reader_obj = csv.reader(file_obj)
        i = 0
        for row in reader_obj:          
          yolov7_annotation_file_name = os.path.splitext(annotation_filename)[0] + "_" + str(row[0]).zfill(7) + ".txt" 
          yolov7_row = _convert_visidrone_row_to_yolov7_row(image_size, row)
          if i == 0:
            print(annotation_filename + "==>" + str(row))          
            print(yolov7_annotation_file_name + "==>" + str(yolov7_row))
          if yolov7_annotation_file_name not in annotation_dict:
            annotation_dict[yolov7_annotation_file_name] = []
          annotation_dict[yolov7_annotation_file_name].append(yolov7_row)
          i += 1
    for k, v in annotation_dict.items():
      with open(str(Path(output_annotations_folder)/k), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(v)



In [13]:
! rm -r /content/VisDroneVideo

In [ ]:
! rm -r /content/VisDroneVideo/VisDrone2019-VID-val-yolov7

In [19]:
def convert_visdrone_data_to_yolov7_format(data_folder_dir, yolov7_output_folder_dir):
  """
    Converts visdrone-VID annotations into yolov7 annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-VID-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
  """
  image_size_dictionary = _visdrone_video_to_yolov7_image_files(data_folder_dir, yolov7_output_folder_dir)
  print(image_size_dictionary)
  _visdrone_video_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir, image_size_dictionary)




In [ ]:
convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-val', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/val')

In [ ]:
convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-train', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/train')

In [ ]:
convert_visdrone_data_to_yolov7_format('/content/VisDroneVideo/VisDrone2019-VID-test-dev', '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/test-dev')

In [28]:
## Create data.yaml file
try:
    with open('/content/VisDroneVideo/VisDrone2019-VID-YOLOv7/data.yaml', 'w') as f:
        f.write('train: ../train/images\n')
        f.writelines('val: ../val/images\n')
        f.writelines('dev: ../test-dev/images\n')
        f.writelines('\n')
        f.writelines('nv: 12\n')
        f.writelines("names: ['ignore', 'pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor', 'others']\n")
except FileNotFoundError:
    print("The 'docs' directory does not exist")


In [29]:
%%time
%%bash
zip -r 'VisDrone2019-VID-YOLOv7.zip' '/content/VisDroneVideo/VisDrone2019-VID-YOLOv7'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 7.85 s, sys: 1.37 s, total: 9.22 s
Wall time: 10min 44s


In [30]:
!cp "/content/VisDrone2019-VID-YOLOv7.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/YOLOv7"

In [ ]:
!python ./yolo_data_utils/yolo_draw_annotation_box.py

In [ ]:
%%time
%%bash
zip -r 'VisDrone2019-VID-YOLOv7-Annotated_Images_Test.zip' '/content/YOLOv7_Annotated_Images'

In [47]:
!cp "/content/VisDrone2019-VID-YOLOv7-Annotated_Images_Test.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/YOLOv7"